In [1]:
import pandas as pd

# Bước 1: Đếm tần suất xuất hiện của user_id
user_freq = pd.read_csv(r"C:\Users\nguye\CODE\TimeSeries\BTL\Data\2019_Data.csv", usecols=["user_id"])["user_id"].value_counts()

# Bước 2: Tính phân vị để chia nhóm
q30 = user_freq.quantile(0.3)
q70 = user_freq.quantile(0.7)
q90 = user_freq.quantile(0.9)

# Bước 3: Gán nhóm A/B/C/D cho từng user
def assign_group(freq):
    if freq >= q90:
        return 'A'
    elif freq >= q70:
        return 'B'
    elif freq >= q30:
        return 'C'
    else:
        return 'D'

user_group = user_freq.apply(assign_group).reset_index()
user_group.columns = ['user_id', 'group']

# Đổi kiểu user_id sang cùng kiểu với file chính (nếu là int/str)
user_group['user_id'] = user_group['user_id'].astype(str)



In [2]:
# Bước 4: Gộp từng chunk và ghi ra 4 file
chunksize = 500_000  # tùy RAM máy bạn, 0.5–1 triệu dòng/lần
reader = pd.read_csv(r"C:\Users\nguye\CODE\TimeSeries\BTL\Data\2019_Data.csv", chunksize=chunksize, dtype={'user_id': str})

# Tạo các file csv đầu ra
output_files = {
    'A': open("2019_GroupA.csv", "w", encoding="utf-8"),
    'B': open("2019_GroupB.csv", "w", encoding="utf-8"),
    'C': open("2019_GroupC.csv", "w", encoding="utf-8"),
    'D': open("2019_GroupD.csv", "w", encoding="utf-8")
}

header_written = {'A': False, 'B': False, 'C': False, 'D': False}

for chunk in reader:
    chunk['user_id'] = chunk['user_id'].astype(str)
    merged = chunk.merge(user_group, on='user_id', how='left')

    for group in ['A', 'B', 'C', 'D']:
        group_data = merged[merged['group'] == group]
        if not group_data.empty:
            group_data.drop(columns='group').to_csv(
                output_files[group],
                header=not header_written[group],
                index=False,
                mode='a'
            )
            header_written[group] = True

# Đóng file
for f in output_files.values():
    f.close()
